In [1]:
from snowflake.snowpark import Session
import snowflake.snowpark.functions as fn
from datetime import date, timedelta

from configparser import ConfigParser

config = ConfigParser()
config.read('config.ini')

connection_parameters: dict[str, int | str] = {
    'user' : config.get('SNOWFLAKE', 'USER'),
    'password' : config.get('SNOWFLAKE', 'PASSWORD'),
    'account' : config.get('SNOWFLAKE', 'ACCOUNT'),
    'database' : config.get('SNOWFLAKE', 'DATABASE'),
    'warehouse' : config.get('SNOWFLAKE', 'WAREHOUSE'),
    'schema' : config.get('SNOWFLAKE', 'SCHEMA'),
    'role' : config.get('SNOWFLAKE', 'ROLE'),
}

session = Session.builder.configs(connection_parameters).create()

In [2]:
from DP_transacciones_total import get_transacciones_dp_total
from general_primeras_compras import get_primeras_compras
from general_get_tiempo import get_tiempo

dp_transacciones = (
    get_transacciones_dp_total(session)
    .filter(fn.col('CANAL').isin('IOS', 'ANDROID'))
)

dp_primeras_compras = (
    get_primeras_compras(dp_transacciones)
    .join(get_tiempo(session), on = 'FECHA')
)

primeras_compras_por_semestre = (
    dp_primeras_compras
    .filter(fn.col('ANIO_ALSEA') == 2023)
    .filter(fn.col('MES_ALSEA').isin(5,6,7,8,9,10))
    .agg(fn.count_distinct('EMAIL'))
)

primeras_compras_por_semestre.to_pandas().to_excel('primeras_compras_MAY-OCT_SOLO_APP.xlsx')

In [3]:
import snowflake.snowpark.functions as fn
from DP_transacciones_total import get_transacciones_dp_total
from general_indicadores import indicadores
from DP_indicadores_por_segmento import indicadores_por_segmento
from general_get_tiempo import get_tiempo

transacciones = (
    get_transacciones_dp_total(session)
    .filter(fn.col('CANAL').isin('IOS', 'ANDROID'))
    .join(get_tiempo(session), on = 'FECHA')
    .filter(fn.col('ANIO_ALSEA') == 2023)
    .filter(fn.col('MES_ALSEA').isin(5,6,7,8,9,10))
)

indicadores_por_segmento(transacciones).to_pandas().to_excel('indicadores_por_segmento_MAY-OCT_SOLO_APP.xlsx')

indicadores(transacciones).to_pandas().to_excel('indicadores_MAY-OCT_SOLO_APP.xlsx')